<a href="https://colab.research.google.com/github/liudiepie/Introduction_to_Machine_Learning/blob/main/Assignment_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Convolutional Neural Network

Image -> <br>
convolution (32 3* 3 filters) -> nonlinearity (ReLU) -> <br>
convolution (32 3* 3 filters) -> nonlinearity (ReLU) -> (2* 2 max pool) <br>
convolution (64 3* 3 filters) -> nonlinearity (ReLU) -> <br>
convolution (64 3* 3 filters) -> nonlinearity (ReLU) -> (2* 2 max pool) -> flatten -> fully connected (256 hidden units) -> nonlinearity (ReLU) -> fully connected (10 hidden units) -> softmax


In [11]:
# Building a custom CNN
import torch.nn as nn

class MNIST_CNN(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(1, 32, kernel_size=5, padding=2)
    self.conv2 = nn.Conv2d(32, 32, kernel_size=5, padding=2)
    self.conv3 = nn.Conv2d(32, 64, kernel_size=5, padding=2)
    self.conv4 = nn.Conv2d(64, 64, kernel_size=5, padding=2)
    self.fc1 = nn.Linear(7*7*64, 256)
    self.fc2 = nn.Linear(256, 10)

  def forward(self, x):
    # conv layer 1
    x = self.conv1(x)
    x = F.relu(x)

    # conv layer 2
    x = self.conv2(x)
    x = F.relu(x)
    x = F.max_pool2d(x, kernel_size=2)

    # conv layer 3
    x = self.conv3(x)
    x = F.relu(x)

    # conv layer 4
    x = self.conv4(x)
    x = F.relu(x)
    x = F.max_pool2d(x, kernel_size=2)

    # fc layer 1 
    x = x.view(-1, 7*7*64)
    x = self.fc1(x)
    x = F.relu(x)

    # fc layer 2
    x = self.fc2(x)
    x = F.softmax(x, dim=None)
    return x

In [3]:
model = MNIST_CNN()
print(model)

MNIST_CNN(
  (conv1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(32, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv3): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv4): Conv2d(64, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (fc1): Linear(in_features=3136, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=10, bias=True)
)


In [4]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from tqdm import tqdm, trange, tqdm_notebook

In [5]:
# Load the data
mnist_train = datasets.MNIST(root="./datasets", train=True, transform=transforms.ToTensor(), download=True)
mnist_test = datasets.MNIST(root="./datasets", train=False, transform=transforms.ToTensor(), download=True)
train_loader = torch.utils.data.DataLoader(mnist_train, batch_size=100, shuffle=True)
test_loader = torch.utils.data.DataLoader(mnist_test, batch_size=100, shuffle=False)


Extracting ./datasets/MNIST/raw/train-images-idx3-ubyte.gz to ./datasets/MNIST/raw




Extracting ./datasets/MNIST/raw/train-labels-idx1-ubyte.gz to ./datasets/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./datasets/MNIST/raw/t10k-images-idx3-ubyte.gz to ./datasets/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./datasets/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./datasets/MNIST/raw



/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [6]:
## Training
# Instantiate model  
model = MNIST_CNN() 

In [8]:
# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001) 

In [12]:
# Iterate through train set minibatchs 
for epoch in trange(3):
    for images, labels in tqdm(train_loader):
        # Zero out the gradients
        optimizer.zero_grad()

        # Forward pass
        x = images 
        y = model(x)
        loss = criterion(y, labels)
        # Backward pass
        loss.backward()
        optimizer.step()

100%|██████████| 3/3 [16:10<00:00, 323.56s/it]


In [13]:
## Testing
correct = 0
total = len(mnist_test)

with torch.no_grad():
    # Iterate through test set minibatchs 
    for images, labels in tqdm(test_loader):
        # Forward pass
        x = images  
        y = model(x)

        predictions = torch.argmax(y, dim=1)
        correct += torch.sum((predictions == labels).float())

print('Test accuracy: {}'.format(correct/total))

100%|██████████| 100/100 [00:17<00:00,  5.63it/s]

Test accuracy: 0.9937999844551086


In [14]:
# Access all weights of a model
params = []
for param in model.parameters():
    params.append(param.view(-1))
params = torch.cat(params)
print(params.shape)

torch.Size([985834])
